In [ ]:
# Instalar bibliotecas necessárias
!pip install torch torchvision
!pip install scikit-image
!pip install matplotlib pillow numpy

# Clonar o repositório do pix2pix
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
%cd pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
import os
from pathlib import Path

# Verificar se GPU está disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando: {device}')

In [ ]:
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Verificar se a pasta existe
caminho_base = '/content/drive/MyDrive/plant_disease'
if os.path.exists(caminho_base):
    print("✓ Pasta plant_disease encontrada!")
else:
    print("✗ Pasta não encontrada. Verifique o nome e localização.")

In [ ]:
from pathlib import Path

def verificar_dataset(caminho_base):
    """
    Verifica e exibe a estrutura do dataset.
    """
    print("=" * 60)
    print("ESTRUTURA DO DATASET")
    print("=" * 60)

    # Verificar pasta de treino
    train_path = Path(caminho_base) / 'train'
    train_images = list(train_path.glob('*.jpg')) + list(train_path.glob('*.png')) + list(train_path.glob('*.jpeg'))
    print(f"\n📁 TREINO (apenas plantas saudáveis):")
    print(f"   Localização: {train_path}")
    print(f"   Imagens: {len(train_images)}")

    # Verificar pasta de teste - saudáveis
    test_healthy_path = Path(caminho_base) / 'test' / 'healthy'
    test_healthy_images = list(test_healthy_path.glob('*.jpg')) + list(test_healthy_path.glob('*.png')) + list(test_healthy_path.glob('*.jpeg'))
    print(f"\n📁 TESTE - SAUDÁVEIS:")
    print(f"   Localização: {test_healthy_path}")
    print(f"   Imagens: {len(test_healthy_images)}")

    # Verificar pasta de teste - doentes
    test_diseased_path = Path(caminho_base) / 'test' / 'diseased'
    test_diseased_images = list(test_diseased_path.glob('*.jpg')) + list(test_diseased_path.glob('*.png')) + list(test_diseased_path.glob('*.jpeg'))
    print(f"\n📁 TESTE - DOENTES:")
    print(f"   Localização: {test_diseased_path}")
    print(f"   Imagens: {len(test_diseased_images)}")

    print("\n" + "=" * 60)
    print(f"✓ Total de imagens: {len(train_images) + len(test_healthy_images) + len(test_diseased_images)}")
    print("=" * 60)

    # Mostrar algumas imagens de exemplo
    if train_images:
        print(f"\nExemplo de arquivos de treino:")
        for img in train_images[:3]:
            print(f"   • {img.name}")

    return {
        'train': train_images,
        'test_healthy': test_healthy_images,
        'test_diseased': test_diseased_images
    }

# Verificar seus dados
dados = verificar_dataset('/content/drive/MyDrive/plant_disease')

In [ ]:
class PlantDataset(Dataset):
    """
    Dataset que carrega imagens de plantas e cria pares (escala de cinza, colorida).
    """
    def __init__(self, lista_imagens, tamanho_imagem=256):
        self.imagens = lista_imagens
        self.tamanho = tamanho_imagem

        # Transformações para redimensionar e normalizar
        self.transform = transforms.Compose([
            transforms.Resize((tamanho_imagem, tamanho_imagem)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.transform_gray = transforms.Compose([
            transforms.Resize((tamanho_imagem, tamanho_imagem)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.imagens)

    def __getitem__(self, idx):
        # Carregar imagem colorida
        img_path = self.imagens[idx]
        img_color = Image.open(img_path).convert('RGB')

        # Converter para escala de cinza
        img_gray = img_color.convert('L')

        # Aplicar transformações
        img_color_tensor = self.transform(img_color)
        img_gray_tensor = self.transform_gray(img_gray)

        return img_gray_tensor, img_color_tensor

# Criar DataLoaders usando suas imagens
train_dataset = PlantDataset(dados['train'])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

print(f"✓ Dataset de treino criado: {len(train_dataset)} imagens")
print(f"✓ DataLoader configurado com batch_size=4")

In [ ]:
def visualizar_amostras(dataset, num_amostras=3):
    """
    Visualiza algumas amostras do dataset.
    """
    fig, axes = plt.subplots(num_amostras, 2, figsize=(10, 4*num_amostras))

    for i in range(num_amostras):
        img_gray, img_color = dataset[i]

        # Desnormalizar
        img_gray_display = (img_gray.squeeze().numpy() + 1) / 2
        img_color_display = (img_color.numpy().transpose(1, 2, 0) + 1) / 2
        img_color_display = np.clip(img_color_display, 0, 1)

        # Plotar
        axes[i, 0].imshow(img_gray_display, cmap='gray')
        axes[i, 0].set_title('Escala de Cinza (Entrada)', fontsize=12)
        axes[i, 0].axis('off')

        axes[i, 1].imshow(img_color_display)
        axes[i, 1].set_title('Colorida (Alvo)', fontsize=12)
        axes[i, 1].axis('off')

    plt.suptitle('Amostras do Dataset de Treino', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualizar suas imagens
visualizar_amostras(train_dataset)

In [ ]:
# Vamos implementar o pix2pix diretamente no Colab
# para ter mais controle sobre o treinamento

class UNetDown(nn.Module):
    """Bloco de downsampling do U-Net"""
    def __init__(self, in_channels, out_channels, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    """Bloco de upsampling do U-Net"""
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x

class GeneratorUNet(nn.Module):
    """Gerador U-Net para pix2pix"""
    def __init__(self, in_channels=1, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        # Encoder (downsampling)
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        # Decoder (upsampling) com skip connections
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)

        return self.final(u7)

class Discriminator(nn.Module):
    """Discriminador PatchGAN para pix2pix"""
    def __init__(self, in_channels=4):  # 1 (gray) + 3 (RGB)
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, 2, 1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img_gray, img_color):
        img_input = torch.cat((img_gray, img_color), 1)
        return self.model(img_input)

# Inicializar modelos
gerador = GeneratorUNet(in_channels=1, out_channels=3).to(device)
discriminador = Discriminator(in_channels=4).to(device)

print("✓ Gerador criado (U-Net com skip connections)")
print("✓ Discriminador criado (PatchGAN)")

In [ ]:
from torch.optim import Adam

# Otimizadores (conforme o artigo)
optimizer_G = Adam(gerador.parameters(), lr=0.000015, betas=(0.9, 0.999))
optimizer_D = Adam(discriminador.parameters(), lr=0.000015, betas=(0.9, 0.999))

# Funções de perda
criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

# Peso para regularização L1 (conforme o artigo)
lambda_L1 = 1

print("✓ Otimizadores configurados (Adam, lr=0.00015)")
print("✓ Funções de perda: BCE + L1")

In [ ]:
from tqdm import tqdm

def treinar_pix2pix(gerador, discriminador, train_loader, num_epochs=150):
    """
    Treina o modelo pix2pix.

    Args:
        gerador: Modelo gerador (U-Net)
        discriminador: Modelo discriminador (PatchGAN)
        train_loader: DataLoader com dados de treino
        num_epochs: Número de épocas (padrão: 150 conforme artigo)
    """
    historico = {
        'loss_G': [],
        'loss_D': [],
        'loss_L1': []
    }

    for epoch in range(num_epochs):
        gerador.train()
        discriminador.train()

        epoch_loss_G = 0
        epoch_loss_D = 0
        epoch_loss_L1 = 0

        # Barra de progresso
        pbar = tqdm(train_loader, desc=f'Época {epoch+1}/{num_epochs}')

        for i, (img_gray, img_color) in enumerate(pbar):
            img_gray = img_gray.to(device)
            img_color = img_color.to(device)

            batch_size = img_gray.size(0)

            # Labels para adversarial loss
            real_label = torch.ones(batch_size, 1, 16, 16).to(device)
            fake_label = torch.zeros(batch_size, 1, 16, 16).to(device)

            # ---------------------
            #  Treinar Gerador
            # ---------------------
            optimizer_G.zero_grad()

            # Gerar imagens falsas
            fake_color = gerador(img_gray)

            # Adversarial loss
            pred_fake = discriminador(img_gray, fake_color)
            loss_GAN = criterion_GAN(pred_fake, real_label)

            # L1 loss
            loss_L1 = criterion_L1(fake_color, img_color)

            # Loss total do gerador
            loss_G = loss_GAN + lambda_L1 * loss_L1
            loss_G.backward()
            optimizer_G.step()

            # ---------------------
            #  Treinar Discriminador
            # ---------------------
            optimizer_D.zero_grad()

            # Loss com imagens reais
            pred_real = discriminador(img_gray, img_color)
            loss_real = criterion_GAN(pred_real, real_label)

            # Loss com imagens falsas
            pred_fake = discriminador(img_gray, fake_color.detach())
            loss_fake = criterion_GAN(pred_fake, fake_label)

            # Loss total do discriminador
            loss_D = (loss_real + loss_fake) * 0.5
            loss_D.backward()
            optimizer_D.step()

            # Acumular losses
            epoch_loss_G += loss_G.item()
            epoch_loss_D += loss_D.item()
            epoch_loss_L1 += loss_L1.item()

            # Atualizar barra de progresso
            pbar.set_postfix({
                'G': f'{loss_G.item():.4f}',
                'D': f'{loss_D.item():.4f}',
                'L1': f'{loss_L1.item():.4f}'
            })

        # Médias da época
        avg_loss_G = epoch_loss_G / len(train_loader)
        avg_loss_D = epoch_loss_D / len(train_loader)
        avg_loss_L1 = epoch_loss_L1 / len(train_loader)

        historico['loss_G'].append(avg_loss_G)
        historico['loss_D'].append(avg_loss_D)
        historico['loss_L1'].append(avg_loss_L1)

        print(f'\nÉpoca {epoch+1}/{num_epochs}:')
        print(f'  Loss Gerador: {avg_loss_G:.4f}')
        print(f'  Loss Discriminador: {avg_loss_D:.4f}')
        print(f'  Loss L1: {avg_loss_L1:.4f}\n')

        # Salvar checkpoint a cada 10 épocas
        if (epoch + 1) % 10 == 0:
            checkpoint_path = f'/content/drive/MyDrive/plant_disease/checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'gerador_state_dict': gerador.state_dict(),
                'discriminador_state_dict': discriminador.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
            }, checkpoint_path)
            print(f'✓ Checkpoint salvo: {checkpoint_path}')

    # Salvar modelo final
    modelo_final = '/content/drive/MyDrive/plant_disease/modelo_final.pth'
    torch.save(gerador.state_dict(), modelo_final)
    print(f'\n✓ Modelo final salvo: {modelo_final}')

    return historico

# INICIAR TREINAMENTO
print("🚀 Iniciando treinamento...")
print(f"   Épocas: 150")
print(f"   Batch size: 4")
print(f"   Imagens de treino: {len(train_dataset)}")
print(f"   Device: {device}\n")

historico = treinar_pix2pix(gerador, discriminador, train_loader, num_epochs=150)

In [ ]:
def plotar_historico(historico):
    """
    Plota as curvas de loss durante o treinamento.
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    epochs = range(1, len(historico['loss_G']) + 1)

    # Loss Gerador e Discriminador
    axes[0].plot(epochs, historico['loss_G'], label='Gerador', color='blue', linewidth=2)
    axes[0].plot(epochs, historico['loss_D'], label='Discriminador', color='red', linewidth=2)
    axes[0].set_xlabel('Época', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Loss Adversarial', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Loss L1
    axes[1].plot(epochs, historico['loss_L1'], label='L1 Loss', color='green', linewidth=2)
    axes[1].set_xlabel('Época', fontsize=12)
    axes[1].set_ylabel('Loss L1', fontsize=12)
    axes[1].set_title('Loss de Reconstrução (L1)', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/plant_disease/historico_treinamento.png', dpi=300)
    plt.show()

    print("✓ Gráfico salvo no Drive")

plotar_historico(historico)

In [ ]:
def carregar_modelo_treinado(caminho_modelo):
    """
    Carrega o modelo treinado do Google Drive.

    Args:
        caminho_modelo: Caminho para o arquivo .pth

    Returns:
        Gerador carregado em modo de avaliação
    """
    gerador = GeneratorUNet(in_channels=1, out_channels=3).to(device)
    gerador.load_state_dict(torch.load(caminho_modelo, map_location=device))
    gerador.eval()

    print("✓ Modelo carregado com sucesso!")
    return gerador

# Para usar depois (após treinar):
gerador = carregar_modelo_treinado('/content/drive/MyDrive/plant_disease/modelo_final.pth')

In [ ]:
from skimage.color import rgb2lab, deltaE_ciede2000

def calcular_ciede2000(img_original, img_reconstruida):
    """
    Calcula a diferença de cor CIEDE2000 entre duas imagens.

    Esta métrica reflete como humanos percebem diferenças de cor.

    Args:
        img_original: Imagem RGB original (numpy array)
        img_reconstruida: Imagem RGB reconstruída (numpy array)

    Returns:
        mapa_diferenca: Mapa de calor com diferenças por pixel
        score_total: Score de anomalia (soma das diferenças)
    """
    # Converter RGB para espaço de cor LAB
    # LAB é melhor para comparar cores como humanos veem
    lab_original = rgb2lab(img_original)
    lab_reconstruida = rgb2lab(img_reconstruida)

    # Calcular diferença CIEDE2000 para cada pixel
    mapa_diferenca = deltaE_ciede2000(lab_original, lab_reconstruida)

    # Score total = soma de todas as diferenças
    score_total = np.sum(mapa_diferenca)

    return mapa_diferenca, score_total

In [ ]:
def diagnosticar_folha(caminho_imagem, gerador, limiar):
    """
    Diagnostica se uma folha está saudável ou doente.

    Args:
        caminho_imagem: Caminho para a imagem da folha
        gerador: Modelo treinado
        limiar: Valor acima do qual a folha é considerada doente

    Returns:
        dict com diagnóstico completo
    """
    # 1. Carregar e preparar imagem
    img_original = Image.open(caminho_imagem).convert('RGB')
    img_original = img_original.resize((256, 256))
    img_array = np.array(img_original) / 255.0

    # 2. Converter para escala de cinza e normalizar
    img_gray = img_original.convert('L')

    transform_gray = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    img_gray_tensor = transform_gray(img_gray).unsqueeze(0).to(device)

    # 3. Reconstruir cores usando o modelo
    gerador.eval()
    with torch.no_grad():
        img_reconstruida_tensor = gerador(img_gray_tensor)

    # 4. Converter de volta para numpy
    img_reconstruida = img_reconstruida_tensor.squeeze().cpu().numpy()
    img_reconstruida = np.transpose(img_reconstruida, (1, 2, 0))
    img_reconstruida = (img_reconstruida + 1) / 2  # Desnormalizar [-1,1] -> [0,1]
    img_reconstruida = np.clip(img_reconstruida, 0, 1)

    # 5. Calcular diferença de cor
    mapa_diferenca, score = calcular_ciede2000(img_array, img_reconstruida)

    # 6. Determinar diagnóstico
    diagnostico = "DOENTE" if score > limiar else "SAUDÁVEL"
    confianca = min(100, (score / limiar) * 100) if diagnostico == "DOENTE" else max(0, 100 - (score / limiar) * 100)

    return {
        'diagnostico': diagnostico,
        'score_anomalia': score,
        'confianca': confianca,
        'imagem_original': img_array,
        'imagem_reconstruida': img_reconstruida,
        'mapa_calor': mapa_diferenca,
        'arquivo': Path(caminho_imagem).name
    }

In [ ]:
def gerar_folha_saudavel(caminho_imagem, gerador):
    """
    Gera uma versão "saudável" da folha reconstruindo suas cores.

    Args:
        caminho_imagem: Caminho para a imagem da folha
        gerador: Modelo treinado

    Returns:
        Imagem numpy array da folha com cores reconstruídas
    """
    # Carregar imagem
    img_original = Image.open(caminho_imagem).convert('RGB')
    img_original = img_original.resize((256, 256))

    # Converter para escala de cinza e normalizar
    img_gray = img_original.convert('L')

    transform_gray = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    img_gray_tensor = transform_gray(img_gray).unsqueeze(0).to(device)

    # Reconstruir cores
    gerador.eval()
    with torch.no_grad():
        img_saudavel_tensor = gerador(img_gray_tensor)

    # Converter para numpy
    img_saudavel = img_saudavel_tensor.squeeze().cpu().numpy()
    img_saudavel = np.transpose(img_saudavel, (1, 2, 0))
    img_saudavel = (img_saudavel + 1) / 2
    img_saudavel = np.clip(img_saudavel, 0, 1)

    return img_saudavel

In [ ]:
def visualizar_indice_cores(resultado_diagnostico, salvar=False, caminho_saida='resultado.png'):
    """
    Cria visualização completa com índice de cores para anomalias.

    Args:
        resultado_diagnostico: Dict retornado por diagnosticar_folha()
        salvar: Se True, salva a figura
        caminho_saida: Onde salvar a figura
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Imagem Original
    axes[0, 0].imshow(resultado_diagnostico['imagem_original'])
    axes[0, 0].set_title('Imagem Original', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')

    # 2. Imagem Reconstruída (Folha "Saudável")
    axes[0, 1].imshow(resultado_diagnostico['imagem_reconstruida'])
    axes[0, 1].set_title('Reconstrução (Folha Saudável)', fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')

    # 3. Mapa de Calor de Anomalias
    im = axes[1, 0].imshow(
        resultado_diagnostico['mapa_calor'],
        cmap='hot',  # Vermelho = mais anômalo
        interpolation='bilinear'
    )
    axes[1, 0].set_title('Mapa de Anomalias (CIEDE2000)', fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')

    # Adicionar barra de cores
    cbar = plt.colorbar(im, ax=axes[1, 0], fraction=0.046, pad=0.04)
    cbar.set_label('Diferença de Cor', rotation=270, labelpad=20)

    # 4. Diagnóstico e Estatísticas
    axes[1, 1].axis('off')

    # Criar texto do diagnóstico
    diagnostico = resultado_diagnostico['diagnostico']
    score = resultado_diagnostico['score_anomalia']
    confianca = resultado_diagnostico['confianca']

    cor_diagnostico = 'red' if diagnostico == "DOENTE" else 'green'

    texto = f"""
    DIAGNÓSTICO: {diagnostico}

    Score de Anomalia: {score:.0f}
    Confiança: {confianca:.1f}%

    Interpretação:
    • Áreas vermelhas = Anomalias fortes
    • Áreas amarelas = Anomalias moderadas
    • Áreas escuras = Tecido saudável

    O modelo foi treinado apenas com
    folhas saudáveis. Áreas doentes
    aparecem em vermelho porque o
    modelo não consegue reconstruir
    suas cores corretamente.
    """

    axes[1, 1].text(
        0.1, 0.5, texto,
        fontsize=12,
        verticalalignment='center',
        bbox=dict(boxstyle='round', facecolor=cor_diagnostico, alpha=0.2)
    )

    plt.suptitle(
        f'Análise Completa - {diagnostico}',
        fontsize=16,
        fontweight='bold',
        color=cor_diagnostico
    )

    plt.tight_layout()

    if salvar:
        plt.savefig(caminho_saida, dpi=300, bbox_inches='tight')
        print(f"✓ Resultado salvo em: {caminho_saida}")

    plt.show()

In [ ]:
# Certifique-se de que o modelo está carregado
gerador = carregar_modelo_treinado('/content/drive/MyDrive/plant_disease/modelo_final.pth')

# 1. Diagnosticar uma imagem de teste doente
caminho_teste = '/content/drive/MyDrive/plant_disease/test/diseased/a976-979 ab_2.jpg'

resultado = diagnosticar_folha(
    caminho_imagem=caminho_teste,
    gerador=gerador,
    limiar=121462  # Ajuste este valor baseado nos seus dados
)

# 2. Visualizar resultado completo
visualizar_indice_cores(resultado, salvar=True,
                        caminho_saida='/content/drive/MyDrive/plant_disease/resultado_diagnostico.png')

# 3. Imprimir resumo
print(f"\n{'='*50}")
print(f"ARQUIVO: {resultado['arquivo']}")
print(f"DIAGNÓSTICO: {resultado['diagnostico']}")
print(f"Score de Anomalia: {resultado['score_anomalia']:.0f}")
print(f"Confiança: {resultado['confianca']:.1f}%")
print(f"{'='*50}\n")

In [ ]:
def encontrar_limiar_otimo(gerador, pasta_saudaveis, pasta_doentes):
    """
    Encontra o limiar ideal testando com imagens saudáveis e doentes.

    Args:
        gerador: Modelo treinado
        pasta_saudaveis: Caminho para pasta com imagens saudáveis
        pasta_doentes: Caminho para pasta com imagens doentes

    Returns:
        Limiar recomendado
    """
    print("🔍 Calculando scores para encontrar limiar ideal...\n")

    # Calcular scores para imagens saudáveis
    scores_saudaveis = []
    imgs_saudaveis = list(Path(pasta_saudaveis).glob('*.jpg')) + \
                     list(Path(pasta_saudaveis).glob('*.png'))

    print(f"Testando {len(imgs_saudaveis)} imagens saudáveis...")
    for img_path in imgs_saudaveis[:20]:  # Testar primeiras 20
        resultado = diagnosticar_folha(str(img_path), gerador, limiar=999999)
        scores_saudaveis.append(resultado['score_anomalia'])

    # Calcular scores para imagens doentes
    scores_doentes = []
    imgs_doentes = list(Path(pasta_doentes).glob('*.jpg')) + \
                   list(Path(pasta_doentes).glob('*.png'))

    print(f"Testando {len(imgs_doentes)} imagens doentes...")
    for img_path in imgs_doentes[:20]:  # Testar primeiras 20
        resultado = diagnosticar_folha(str(img_path), gerador, limiar=999999)
        scores_doentes.append(resultado['score_anomalia'])

    # Calcular estatísticas
    media_saudavel = np.mean(scores_saudaveis)
    std_saudavel = np.std(scores_saudaveis)
    media_doente = np.mean(scores_doentes)
    std_doente = np.std(scores_doentes)

    # Limiar = ponto médio entre as médias
    limiar_recomendado = (media_saudavel + media_doente) / 2

    # Visualizar distribuições
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(scores_saudaveis, bins=20, alpha=0.7, color='green', label='Saudáveis')
    plt.hist(scores_doentes, bins=20, alpha=0.7, color='red', label='Doentes')
    plt.axvline(limiar_recomendado, color='blue', linestyle='--', linewidth=2, label=f'Limiar: {limiar_recomendado:.0f}')
    plt.xlabel('Score de Anomalia')
    plt.ylabel('Frequência')
    plt.title('Distribuição dos Scores')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.boxplot([scores_saudaveis, scores_doentes], labels=['Saudáveis', 'Doentes'])
    plt.ylabel('Score de Anomalia')
    plt.title('Comparação dos Scores')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/plant_disease/analise_limiar.png', dpi=300)
    plt.show()

    print(f"\n{'='*60}")
    print(f"ESTATÍSTICAS:")
    print(f"  Saudáveis: média={media_saudavel:.0f}, std={std_saudavel:.0f}")
    print(f"  Doentes:   média={media_doente:.0f}, std={std_doente:.0f}")
    print(f"\n✓ LIMIAR RECOMENDADO: {limiar_recomendado:.0f}")
    print(f"{'='*60}\n")

    return limiar_recomendado

# Encontrar limiar ideal
limiar_otimo = encontrar_limiar_otimo(
    gerador,
    pasta_saudaveis='/content/drive/MyDrive/plant_disease/test/healthy',
    pasta_doentes='/content/drive/MyDrive/plant_disease/test/diseased'
)

In [ ]:
def processar_lote_completo(gerador, dados, limiar):
    """
    Processa todas as imagens de teste e gera relatório.

    Args:
        gerador: Modelo treinado
        dados: Dict retornado pela função verificar_dataset
        limiar: Valor de limiar para classificação

    Returns:
        DataFrame com resultados
    """
    import pandas as pd

    resultados = []

    # Criar pasta para salvar resultados
    pasta_resultados = '/content/drive/MyDrive/plant_disease/resultados'
    os.makedirs(pasta_resultados, exist_ok=True)
    os.makedirs(f'{pasta_resultados}/healthy', exist_ok=True)
    os.makedirs(f'{pasta_resultados}/diseased', exist_ok=True)

    print("🔬 PROCESSANDO IMAGENS DE TESTE\n")

    # Processar imagens saudáveis
    print("📗 Processando folhas saudáveis...")
    for img_path in tqdm(dados['test_healthy']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'SAUDÁVEL',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'SAUDÁVEL'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/healthy/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Processar imagens doentes
    print("\n📕 Processando folhas doentes...")
    for img_path in tqdm(dados['test_diseased']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'DOENTE',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'DOENTE'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/diseased/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Criar DataFrame
    df = pd.DataFrame(resultados)

    # Calcular métricas
    acuracia = (df['correto'].sum() / len(df)) * 100

    precisao_doente = (df[(df['diagnostico'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                       df[df['diagnostico'] == 'DOENTE'].shape[0] * 100) if df[df['diagnostico'] == 'DOENTE'].shape[0] > 0 else 0

    recall_doente = (df[(df['tipo_real'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                     df[df['tipo_real'] == 'DOENTE'].shape[0] * 100)

    # Salvar CSV
    df.to_csv(f'{pasta_resultados}/resultados_completos.csv', index=False)

    print(f"\n{'='*60}")
    print(f"RESULTADOS FINAIS:")
    print(f"  Total de imagens: {len(df)}")
    print(f"  Acurácia: {acuracia:.2f}%")
    print(f"  Precisão (doentes): {precisao_doente:.2f}%")
    print(f"  Recall (doentes): {recall_doente:.2f}%")
    print(f"\n✓ Resultados salvos em: {pasta_resultados}")
    print(f"{'='*60}\n")

    return df

# Processar tudo
df_resultados = processar_lote_completo(gerador, dados, limiar_otimo)

# Visualizar matriz de confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(df_resultados['tipo_real'], df_resultados['diagnostico'],
                      labels=['SAUDÁVEL', 'DOENTE'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['SAUDÁVEL', 'DOENTE'],
            yticklabels=['SAUDÁVEL', 'DOENTE'])
plt.ylabel('Tipo Real')
plt.xlabel('Diagnóstico')
plt.title('Matriz de Confusão', fontsize=14, fontweight='bold')
plt.savefig('/content/drive/MyDrive/plant_disease/matriz_confusao.png', dpi=300)
plt.show()

In [ ]:
def encontrar_limiar_otimo(gerador, pasta_saudaveis, pasta_doentes):
    """
    Encontra o limiar ideal testando com imagens saudáveis e doentes.

    Args:
        gerador: Modelo treinado
        pasta_saudaveis: Caminho para pasta com imagens saudáveis
        pasta_doentes: Caminho para pasta com imagens doentes

    Returns:
        Limiar recomendado
    """
    print("🔍 Calculando scores para encontrar limiar ideal...\n")

    # Calcular scores para imagens saudáveis
    scores_saudaveis = []
    imgs_saudaveis = list(Path(pasta_saudaveis).glob('*.jpg')) + \
                     list(Path(pasta_saudaveis).glob('*.png'))

    print(f"Testando {len(imgs_saudaveis)} imagens saudáveis...")
    for img_path in imgs_saudaveis[:50]:  # Testar primeiras 20
        resultado = diagnosticar_folha(str(img_path), gerador, limiar=999999)
        scores_saudaveis.append(resultado['score_anomalia'])

    # Calcular scores para imagens doentes
    scores_doentes = []
    imgs_doentes = list(Path(pasta_doentes).glob('*.jpg')) + \
                   list(Path(pasta_doentes).glob('*.png'))

    print(f"Testando {len(imgs_doentes)} imagens doentes...")
    for img_path in imgs_doentes[:100]:  # Testar primeiras 20
        resultado = diagnosticar_folha(str(img_path), gerador, limiar=999999)
        scores_doentes.append(resultado['score_anomalia'])

    # Calcular estatísticas
    media_saudavel = np.mean(scores_saudaveis)
    std_saudavel = np.std(scores_saudaveis)
    media_doente = np.mean(scores_doentes)
    std_doente = np.std(scores_doentes)

    # Limiar = ponto médio entre as médias
    limiar_recomendado = (media_saudavel + media_doente) / 2

    # Visualizar distribuições
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(scores_saudaveis, bins=20, alpha=0.7, color='green', label='Saudáveis')
    plt.hist(scores_doentes, bins=20, alpha=0.7, color='red', label='Doentes')
    plt.axvline(limiar_recomendado, color='blue', linestyle='--', linewidth=2, label=f'Limiar: {limiar_recomendado:.0f}')
    plt.xlabel('Score de Anomalia')
    plt.ylabel('Frequência')
    plt.title('Distribuição dos Scores')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.boxplot([scores_saudaveis, scores_doentes], labels=['Saudáveis', 'Doentes'])
    plt.ylabel('Score de Anomalia')
    plt.title('Comparação dos Scores')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/plant_disease/analise_limiar.png', dpi=300)
    plt.show()

    print(f"\n{'='*60}")
    print(f"ESTATÍSTICAS:")
    print(f"  Saudáveis: média={media_saudavel:.0f}, std={std_saudavel:.0f}")
    print(f"  Doentes:   média={media_doente:.0f}, std={std_doente:.0f}")
    print(f"\n✓ LIMIAR RECOMENDADO: {limiar_recomendado:.0f}")
    print(f"{'='*60}\n")

    return limiar_recomendado

# Encontrar limiar ideal
limiar_otimo = encontrar_limiar_otimo(
    gerador,
    pasta_saudaveis='/content/drive/MyDrive/plant_disease/test/healthy',
    pasta_doentes='/content/drive/MyDrive/plant_disease/test/diseased'
)

In [ ]:
def processar_lote_completo(gerador, dados, limiar=121462):
    """
    Processa todas as imagens de teste e gera relatório.

    Args:
        gerador: Modelo treinado
        dados: Dict retornado pela função verificar_dataset
        limiar: Valor de limiar para classificação

    Returns:
        DataFrame com resultados
    """
    import pandas as pd

    resultados = []

    # Criar pasta para salvar resultados
    pasta_resultados = '/content/drive/MyDrive/plant_disease/resultados'
    os.makedirs(pasta_resultados, exist_ok=True)
    os.makedirs(f'{pasta_resultados}/healthy', exist_ok=True)
    os.makedirs(f'{pasta_resultados}/diseased', exist_ok=True)

    print("🔬 PROCESSANDO IMAGENS DE TESTE\n")

    # Processar imagens saudáveis
    print("📗 Processando folhas saudáveis...")
    for img_path in tqdm(dados['test_healthy']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'SAUDÁVEL',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'SAUDÁVEL'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/healthy/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Processar imagens doentes
    print("\n📕 Processando folhas doentes...")
    for img_path in tqdm(dados['test_diseased']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'DOENTE',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'DOENTE'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/diseased/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Criar DataFrame
    df = pd.DataFrame(resultados)

    # Calcular métricas
    acuracia = (df['correto'].sum() / len(df)) * 100

    precisao_doente = (df[(df['diagnostico'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                       df[df['diagnostico'] == 'DOENTE'].shape[0] * 100) if df[df['diagnostico'] == 'DOENTE'].shape[0] > 0 else 0

    recall_doente = (df[(df['tipo_real'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                     df[df['tipo_real'] == 'DOENTE'].shape[0] * 100)

    # Salvar CSV
    df.to_csv(f'{pasta_resultados}/resultados_completos.csv', index=False)

    print(f"\n{'='*60}")
    print(f"RESULTADOS FINAIS:")
    print(f"  Total de imagens: {len(df)}")
    print(f"  Acurácia: {acuracia:.2f}%")
    print(f"  Precisão (doentes): {precisao_doente:.2f}%")
    print(f"  Recall (doentes): {recall_doente:.2f}%")
    print(f"\n✓ Resultados salvos em: {pasta_resultados}")
    print(f"{'='*60}\n")

    return df

# Processar tudo
df_resultados = processar_lote_completo(gerador, dados, limiar_otimo)

# Visualizar matriz de confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(df_resultados['tipo_real'], df_resultados['diagnostico'],
                      labels=['SAUDÁVEL', 'DOENTE'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['SAUDÁVEL', 'DOENTE'],
            yticklabels=['SAUDÁVEL', 'DOENTE'])
plt.ylabel('Tipo Real')
plt.xlabel('Diagnóstico')
plt.title('Matriz de Confusão', fontsize=14, fontweight='bold')
plt.savefig('/content/drive/MyDrive/plant_disease/matriz_confusao.png', dpi=300)
plt.show()

In [ ]:
def processar_lote_completo(gerador, dados, limiar):
    """
    Processa todas as imagens de teste e gera relatório.

    Args:
        gerador: Modelo treinado
        dados: Dict retornado pela função verificar_dataset
        limiar: Valor de limiar para classificação

    Returns:
        DataFrame com resultados
    """
    import pandas as pd

    resultados = []

    # Criar pasta para salvar resultados
    pasta_resultados = '/content/drive/MyDrive/plant_disease/resultados'
    os.makedirs(pasta_resultados, exist_ok=True)
    os.makedirs(f'{pasta_resultados}/healthy', exist_ok=True)
    os.makedirs(f'{pasta_resultados}/diseased', exist_ok=True)

    print("🔬 PROCESSANDO IMAGENS DE TESTE\n")

    # Processar imagens saudáveis
    print("📗 Processando folhas saudáveis...")
    for img_path in tqdm(dados['test_healthy']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'SAUDÁVEL',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'SAUDÁVEL'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/healthy/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Processar imagens doentes
    print("\n📕 Processando folhas doentes...")
    for img_path in tqdm(dados['test_diseased']):
        resultado = diagnosticar_folha(str(img_path), gerador, limiar)

        resultados.append({
            'arquivo': resultado['arquivo'],
            'tipo_real': 'DOENTE',
            'diagnostico': resultado['diagnostico'],
            'score': resultado['score_anomalia'],
            'confianca': resultado['confianca'],
            'correto': resultado['diagnostico'] == 'DOENTE'
        })

        # Salvar visualização
        visualizar_indice_cores(
            resultado,
            salvar=True,
            caminho_saida=f"{pasta_resultados}/diseased/{resultado['arquivo'].replace('.jpg', '_analise.png')}"
        )
        plt.close()

    # Criar DataFrame
    df = pd.DataFrame(resultados)

    # Calcular métricas
    acuracia = (df['correto'].sum() / len(df)) * 100

    precisao_doente = (df[(df['diagnostico'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                       df[df['diagnostico'] == 'DOENTE'].shape[0] * 100) if df[df['diagnostico'] == 'DOENTE'].shape[0] > 0 else 0

    recall_doente = (df[(df['tipo_real'] == 'DOENTE') & (df['correto'] == True)].shape[0] /
                     df[df['tipo_real'] == 'DOENTE'].shape[0] * 100)

    # Salvar CSV
    df.to_csv(f'{pasta_resultados}/resultados_completos.csv', index=False)

    print(f"\n{'='*60}")
    print(f"RESULTADOS FINAIS:")
    print(f"  Total de imagens: {len(df)}")
    print(f"  Acurácia: {acuracia:.2f}%")
    print(f"  Precisão (doentes): {precisao_doente:.2f}%")
    print(f"  Recall (doentes): {recall_doente:.2f}%")
    print(f"\n✓ Resultados salvos em: {pasta_resultados}")
    print(f"{'='*60}\n")

    return df

# Processar tudo
df_resultados = processar_lote_completo(gerador, dados, limiar_otimo)

# Visualizar matriz de confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(df_resultados['tipo_real'], df_resultados['diagnostico'],
                      labels=['SAUDÁVEL', 'DOENTE'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['SAUDÁVEL', 'DOENTE'],
            yticklabels=['SAUDÁVEL', 'DOENTE'])
plt.ylabel('Tipo Real')
plt.xlabel('Diagnóstico')
plt.title('Matriz de Confusão', fontsize=14, fontweight='bold')
plt.savefig('/content/drive/MyDrive/plant_disease/matriz_confusao.png', dpi=300)
plt.show()

# Task
Implement Grad-CAM by defining functions to register forward and backward hooks on the Generator model, enabling extraction of feature maps and gradients from the last convolutional layer. Then, apply these to calculate Grad-CAM heatmaps for image classification.

## Implementar Grad-CAM

### Subtask:
Implementar as funções necessárias para calcular o Grad-CAM no modelo Gerador. Isso envolve registrar hooks para extrair mapas de características e gradientes.


## Implementando Grad-CAM para Análise de Anomalias

Para entender **onde** o modelo Gerador detecta anomalias nas folhas (ou seja, onde ele tem dificuldade em reconstruir a cor original), vamos implementar o **Grad-CAM (Gradient-weighted Class Activation Mapping)**. Embora o Grad-CAM seja tipicamente usado para modelos de classificação, podemos adaptá-lo para nosso cenário de detecção de anomalias.

**Como funciona no nosso contexto:**
1.  O modelo Gerador recebe uma imagem em escala de cinza e tenta colorizá-la (produzir a imagem 'saudável' reconstruída).
2.  Uma 'anomalia' é detectada quando a imagem reconstruída é significativamente diferente da imagem original colorida. A métrica `CIEDE2000` quantifica essa diferença.
3.  Para o Grad-CAM, precisamos de um 'score' ou 'loss' para retropropagar. Em nosso caso, podemos usar a `Loss L1` (ou a `Loss GAN` do gerador) em relação à imagem colorida real como o 'alvo' para a retropropagação.
4.  O Grad-CAM nos mostrará quais partes da camada de características final do Gerador foram mais ativadas para produzir a diferença na reconstrução (ou seja, onde a Loss L1 foi maior).

Vamos focar em uma camada específica do decoder do Gerador, pois é lá que as informações visuais de alto nível são combinadas para reconstruir a imagem. O objetivo é visualizar as regiões que mais contribuíram para a 'dificuldade' de colorização do Gerador, indicando áreas anômalas.

**Reasoning**:
Now, I will implement the `GradCAM` class as described in the instructions, setting up forward and backward hooks to capture feature maps and gradients from a specified layer of the generator model.



In [ ]:
import torch.nn.functional as F

class GradCAM:
    """
    Implementa Grad-CAM para visualizar as regiões mais ativas no Gerador.
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.feature_maps = None
        self.gradients = None

        # Registrar hooks
        self.target_layer.register_forward_hook(self._save_feature_maps_hook)
        self.target_layer.register_full_backward_hook(self._save_gradients_hook)

    def _save_feature_maps_hook(self, module, input, output):
        self.feature_maps = output

    def _save_gradients_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate_heatmap(self, input_tensor, target_output):
        self.model.eval()
        # Limpar gradientes anteriores
        if self.feature_maps is not None:
            self.feature_maps.retain_grad()

        # Realizar forward pass
        output = self.model(input_tensor)

        # Zero os gradientes do modelo antes do backward pass
        self.model.zero_grad()

        # Calcular a loss em relação ao 'alvo' para o backward pass
        # Usamos L1 Loss entre a saída do gerador e a imagem colorida real
        loss = F.l1_loss(output, target_output) # target_output aqui seria a imagem colorida original
        loss.backward()

        # Extrair feature maps e gradients capturados pelos hooks
        feature_maps = self.feature_maps.cpu().data.numpy()[0]
        gradients = self.gradients.cpu().data.numpy()[0]

        # Pooling global dos gradientes (média)
        pooled_gradients = np.mean(gradients, axis=(1, 2))

        # Ponderar os feature maps pelos gradientes médios
        for i, w in enumerate(pooled_gradients):
            feature_maps[i, :, :] = w * feature_maps[i, :, :]

        # Criar o heatmap
        heatmap = np.sum(feature_maps, axis=0)
        heatmap = np.maximum(heatmap, 0) # Aplicar ReLU

        # Normalizar heatmap
        if np.max(heatmap) > 0:
            heatmap /= np.max(heatmap)

        # Redimensionar para o tamanho da imagem original (256x256)
        heatmap = F.interpolate(
            torch.from_numpy(heatmap).unsqueeze(0).unsqueeze(0),
            size=(input_tensor.shape[2], input_tensor.shape[3]),
            mode='bilinear',
            align_corners=False
        ).squeeze().numpy()

        return heatmap

print("✓ Classe GradCAM implementada com sucesso.")

## Aplicar Grad-CAM em Imagens de Teste

### Subtask:
Selecionar imagens de teste (saudáveis e doentes) e aplicar a lógica do Grad-CAM para gerar mapas de calor que destacam as regiões mais importantes para o diagnóstico do modelo.


**Reasoning**:
First, I will choose a target layer within the generator model for Grad-CAM. Then, I will instantiate the GradCAM class with the generator and the selected layer. After that, I will select one healthy and one diseased image from the test datasets. Finally, for each selected image, I will prepare the input tensors, generate the Grad-CAM heatmap, and store all necessary outputs for subsequent visualization.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Criar pasta para salvar resultados do Grad-CAM
pasta_gradcam_results = '/content/drive/MyDrive/plant_disease/gradcam_results/'
os.makedirs(pasta_gradcam_results, exist_ok=True)
print(f"✓ Pasta de resultados Grad-CAM criada em: {pasta_gradcam_results}")

for img_type, result in gradcam_results.items():
    original_img = result['original_img']
    generated_img = result['generated_img']
    heatmap = result['heatmap']
    filename = result['filename']

    # Redimensionar heatmap para o tamanho da imagem original (já feito no generate_heatmap, mas garantindo)
    # heatmap_resized = np.array(Image.fromarray((heatmap * 255).astype(np.uint8)).resize(original_img.shape[1::-1], Image.BILINEAR)) / 255.0
    # A linha acima não é necessária porque o heatmap já foi interpolado para o tamanho correto dentro de generate_heatmap
    heatmap_resized = heatmap # Usamos o heatmap já redimensionado

    # Criar a sobreposição
    overlay = original_img.copy()
    # Convert original_img to 3 channels if it's not already, for consistent overlaying
    if len(overlay.shape) == 2:
        overlay = np.stack([overlay, overlay, overlay], axis=-1)

    # Aplicar o heatmap como uma máscara de calor no canal vermelho, ou como alpha
    # Usando plt.imshow para a sobreposição para melhor controle de cmap e alpha

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle(f'Grad-CAM Análise - {img_type.upper()} ({filename})', fontsize=16, fontweight='bold')

    # 1. Imagem Original
    axes[0].imshow(original_img)
    axes[0].set_title('Imagem Original')
    axes[0].axis('off')

    # 2. Imagem Gerada pelo Modelo
    axes[1].imshow(generated_img)
    axes[1].set_title('Imagem Gerada (Reconstruída)')
    axes[1].axis('off')

    # 3. Grad-CAM Heatmap
    im = axes[2].imshow(heatmap_resized, cmap='jet', alpha=0.7)
    axes[2].set_title('Grad-CAM Heatmap')
    axes[2].axis('off')
    plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)

    # 4. Sobreposição do Heatmap na Imagem Original
    axes[3].imshow(original_img)
    axes[3].imshow(heatmap_resized, cmap='jet', alpha=0.5) # Alpha para visualização da imagem subjacente
    axes[3].set_title('Heatmap sobre Original')
    axes[3].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar layout para não sobrepor o título principal

    # Salvar a figura
    save_path = os.path.join(pasta_gradcam_results, f'gradcam_{img_type}_{filename.replace(".jpg", ".png").replace(".jpeg", ".png")}')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"✓ Gráfico Grad-CAM salvo em: {save_path}")
    plt.show()

print("✓ Visualização dos resultados Grad-CAM concluída.")

In [ ]:
import numpy as np
from PIL import Image
from torchvision import transforms

# 1. Escolher uma camada convolucional adequada do modelo gerador
# A instrução sugere 'gerador.up7.model[0]' ou 'gerador.down8.model[0]'.
# Vamos usar a primeira camada convolucional do bloco 'up7' do gerador.
target_layer = gerador.up7.model[0] # Ou gerador.down8.model[0] se preferir uma camada do encoder

# 2. Crie uma instância da classe GradCAM
grad_cam_instance = GradCAM(gerador, target_layer)
print(f"✓ Instância GradCAM criada com camada alvo: {target_layer}")

# 3. Selecionar uma imagem de folha saudável e uma imagem de folha doente
# Usando os dados já carregados de `dados`
healthy_img_path = dados['test_healthy'][0] # Primeira imagem saudável
diseased_img_path = dados['test_diseased'][0] # Primeira imagem doente

print(f"\nImagens selecionadas:\n  Saudável: {healthy_img_path.name}\n  Doente:   {diseased_img_path.name}")

# Prepare transforms for input and target
# For input_tensor (grayscale) - this is what the generator takes
transform_gray_input = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# For target_output (original color) - this is for L1 loss calculation in GradCAM
transform_color_target = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Armazenar resultados para visualização posterior
gradcam_results = {}

for img_type, path in [('healthy', healthy_img_path), ('diseased', diseased_img_path)]:
    print(f"\nProcessando imagem {img_type}: {path.name}")

    # a. Carregue a imagem original e redimensione-a para 256x256 pixels.
    original_img_pil = Image.open(path).convert('RGB')
    original_img_numpy = np.array(original_img_pil.resize((256, 256))) / 255.0 # Normalizado para [0,1] para exibição

    # b. Converta a imagem original para escala de cinza e aplique as transformações
    img_gray_pil = original_img_pil.convert('L')
    input_tensor = transform_gray_input(img_gray_pil).unsqueeze(0).to(device)

    # c. Aplique as transformações na imagem colorida original para criar o target_output
    target_output_tensor = transform_color_target(original_img_pil).unsqueeze(0).to(device)

    # d. Gere o mapa de calor do Grad-CAM
    heatmap = grad_cam_instance.generate_heatmap(input_tensor, target_output_tensor)

    # Gerar a imagem colorida pelo gerador (saída do gerador para input_tensor)
    gerador.eval()
    with torch.no_grad():
        generated_img_tensor = gerador(input_tensor)
    generated_img_numpy = generated_img_tensor.squeeze().cpu().numpy()
    generated_img_numpy = np.transpose(generated_img_numpy, (1, 2, 0))
    generated_img_numpy = (generated_img_numpy + 1) / 2 # Desnormalizar [-1,1] -> [0,1]
    generated_img_numpy = np.clip(generated_img_numpy, 0, 1)

    # e. Armazene a imagem original, a imagem de entrada em escala de cinza, a imagem reconstruída e o mapa de calor
    gradcam_results[img_type] = {
        'original_img': original_img_numpy,
        'gray_input_img': np.array(img_gray_pil.resize((256,256))) / 255.0,
        'generated_img': generated_img_numpy,
        'heatmap': heatmap,
        'filename': path.name
    }

print("✓ Resultados do Grad-CAM calculados e armazenados para as imagens selecionadas.")

**Reasoning**:
The error `RuntimeError: can't retain_grad on Tensor that has requires_grad=False` indicates that `retain_grad()` was called on a tensor (`self.feature_maps`) that does not track gradients. I need to ensure that the feature maps obtained from the forward hook have `requires_grad=True` so that gradients can be computed and retained for them during the backward pass. I will modify the `_save_feature_maps_hook` to set `requires_grad_()` on the output tensor.



In [ ]:
import torch.nn.functional as F

class GradCAM:
    """
    Implementa Grad-CAM para visualizar as regiões mais ativas no Gerador.
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.feature_maps = None
        self.gradients = None

        # Registrar hooks
        self.target_layer.register_forward_hook(self._save_feature_maps_hook)
        self.target_layer.register_full_backward_hook(self._save_gradients_hook)

    def _save_feature_maps_hook(self, module, input, output):
        # Ensure feature_maps require gradients to call .retain_grad() later
        output.requires_grad_(True)
        self.feature_maps = output

    def _save_gradients_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate_heatmap(self, input_tensor, target_output):
        self.model.eval()

        # Realizar forward pass
        output = self.model(input_tensor)

        # Zero os gradientes do modelo antes do backward pass
        self.model.zero_grad()

        # Calcular a loss em relação ao 'alvo' para o backward pass
        # Usamos L1 Loss entre a saída do gerador e a imagem colorida real
        loss = F.l1_loss(output, target_output) # target_output aqui seria a imagem colorida original
        loss.backward()

        # Extrair feature maps e gradients capturados pelos hooks
        feature_maps = self.feature_maps.cpu().data.numpy()[0]
        gradients = self.gradients.cpu().data.numpy()[0]

        # Pooling global dos gradientes (média)
        pooled_gradients = np.mean(gradients, axis=(1, 2))

        # Ponderar os feature maps pelos gradientes médios
        for i, w in enumerate(pooled_gradients):
            feature_maps[i, :, :] = w * feature_maps[i, :, :]

        # Criar o heatmap
        heatmap = np.sum(feature_maps, axis=0)
        heatmap = np.maximum(heatmap, 0) # Aplicar ReLU

        # Normalizar heatmap
        if np.max(heatmap) > 0:
            heatmap /= np.max(heatmap)

        # Redimensionar para o tamanho da imagem original (256x256)
        heatmap = F.interpolate(
            torch.from_numpy(heatmap).unsqueeze(0).unsqueeze(0),
            size=(input_tensor.shape[2], input_tensor.shape[3]),
            mode='bilinear',
            align_corners=False
        ).squeeze().numpy()

        return heatmap

print("✓ Classe GradCAM implementada com sucesso.")

**Reasoning**:
The `GradCAM` class has been updated to correctly handle gradient retention. I will now re-run the previous code block to apply the corrected Grad-CAM implementation to the selected healthy and diseased test images, generate their heatmaps, and store the results for visualization as per the subtask instructions.



In [ ]:
import numpy as np
from PIL import Image
from torchvision import transforms

# 1. Escolher uma camada convolucional adequada do modelo gerador
# A instrução sugere 'gerador.up7.model[0]' ou 'gerador.down8.model[0]'.
# Vamos usar a primeira camada convolucional do bloco 'up7' do gerador.
target_layer = gerador.up7.model[0] # Ou gerador.down8.model[0] se preferir uma camada do encoder

# 2. Crie uma instância da classe GradCAM
grad_cam_instance = GradCAM(gerador, target_layer)
print(f"✓ Instância GradCAM criada com camada alvo: {target_layer}")

# 3. Selecionar uma imagem de folha saudável e uma imagem de folha doente
# Usando os dados já carregados de `dados`
healthy_img_path = dados['test_healthy'][0] # Primeira imagem saudável
diseased_img_path = dados['test_diseased'][0] # Primeira imagem doente

print(f"\nImagens selecionadas:\n  Saudável: {healthy_img_path.name}\n  Doente:   {diseased_img_path.name}")

# Prepare transforms for input and target
# For input_tensor (grayscale) - this is what the generator takes
transform_gray_input = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# For target_output (original color) - this is for L1 loss calculation in GradCAM
transform_color_target = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Armazenar resultados para visualização posterior
gradcam_results = {}

for img_type, path in [('healthy', healthy_img_path), ('diseased', diseased_img_path)]:
    print(f"\nProcessando imagem {img_type}: {path.name}")

    # a. Carregue a imagem original e redimensione-a para 256x256 pixels.
    original_img_pil = Image.open(path).convert('RGB')
    original_img_numpy = np.array(original_img_pil.resize((256, 256))) / 255.0 # Normalizado para [0,1] para exibição

    # b. Converta a imagem original para escala de cinza e aplique as transformações
    img_gray_pil = original_img_pil.convert('L')
    input_tensor = transform_gray_input(img_gray_pil).unsqueeze(0).to(device)

    # c. Aplique as transformações na imagem colorida original para criar o target_output
    target_output_tensor = transform_color_target(original_img_pil).unsqueeze(0).to(device)

    # d. Gere o mapa de calor do Grad-CAM
    heatmap = grad_cam_instance.generate_heatmap(input_tensor, target_output_tensor)

    # Gerar a imagem colorida pelo gerador (saída do gerador para input_tensor)
    gerador.eval()
    with torch.no_grad():
        generated_img_tensor = gerador(input_tensor)
    generated_img_numpy = generated_img_tensor.squeeze().cpu().numpy()
    generated_img_numpy = np.transpose(generated_img_numpy, (1, 2, 0))
    generated_img_numpy = (generated_img_numpy + 1) / 2 # Desnormalizar [-1,1] -> [0,1]
    generated_img_numpy = np.clip(generated_img_numpy, 0, 1)

    # e. Armazene a imagem original, a imagem de entrada em escala de cinza, a imagem reconstruída e o mapa de calor
    gradcam_results[img_type] = {
        'original_img': original_img_numpy,
        'gray_input_img': np.array(img_gray_pil.resize((256,256))) / 255.0,
        'generated_img': generated_img_numpy,
        'heatmap': heatmap,
        'filename': path.name
    }

print("✓ Resultados do Grad-CAM calculados e armazenados para as imagens selecionadas.")

## Visualizar Resultados do Grad-CAM

### Subtask:
Plotar as imagens originais, os mapas de calor do Grad-CAM e a sobreposição desses mapas nas imagens originais para uma análise visual das regiões de anomalia.


**Reasoning**:
I need to import `matplotlib.pyplot` and `seaborn` for plotting. Although `seaborn` is not explicitly used for plotting in the instructions, it's good practice to import it alongside `matplotlib` for enhanced aesthetics if needed later. Then, I will iterate through the `gradcam_results` to generate and display visualizations for each image type as specified.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Criar pasta para salvar resultados do Grad-CAM
pasta_gradcam_results = '/content/drive/MyDrive/plant_disease/gradcam_results/'
os.makedirs(pasta_gradcam_results, exist_ok=True)
print(f"✓ Pasta de resultados Grad-CAM criada em: {pasta_gradcam_results}")

for img_type, result in gradcam_results.items():
    original_img = result['original_img']
    generated_img = result['generated_img']
    heatmap = result['heatmap']
    filename = result['filename']

    # Redimensionar heatmap para o tamanho da imagem original (já feito no generate_heatmap, mas garantindo)
    # heatmap_resized = np.array(Image.fromarray((heatmap * 255).astype(np.uint8)).resize(original_img.shape[1::-1], Image.BILINEAR)) / 255.0
    # A linha acima não é necessária porque o heatmap já foi interpolado para o tamanho correto dentro de generate_heatmap
    heatmap_resized = heatmap # Usamos o heatmap já redimensionado

    # Criar a sobreposição
    overlay = original_img.copy()
    # Convert original_img to 3 channels if it's not already, for consistent overlaying
    if len(overlay.shape) == 2:
        overlay = np.stack([overlay, overlay, overlay], axis=-1)

    # Aplicar o heatmap como uma máscara de calor no canal vermelho, ou como alpha
    # Usando plt.imshow para a sobreposição para melhor controle de cmap e alpha

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle(f'Grad-CAM Análise - {img_type.upper()} ({filename})', fontsize=16, fontweight='bold')

    # 1. Imagem Original
    axes[0].imshow(original_img)
    axes[0].set_title('Imagem Original')
    axes[0].axis('off')

    # 2. Imagem Gerada pelo Modelo
    axes[1].imshow(generated_img)
    axes[1].set_title('Imagem Gerada (Reconstruída)')
    axes[1].axis('off')

    # 3. Grad-CAM Heatmap
    im = axes[2].imshow(heatmap_resized, cmap='jet', alpha=0.7)
    axes[2].set_title('Grad-CAM Heatmap')
    axes[2].axis('off')
    plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)

    # 4. Sobreposição do Heatmap na Imagem Original
    axes[3].imshow(original_img)
    axes[3].imshow(heatmap_resized, cmap='jet', alpha=0.5) # Alpha para visualização da imagem subjacente
    axes[3].set_title('Heatmap sobre Original')
    axes[3].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar layout para não sobrepor o título principal

    # Salvar a figura
    save_path = os.path.join(pasta_gradcam_results, f'gradcam_{img_type}_{filename.replace(".jpg", ".png").replace(".jpeg", ".png")}')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"✓ Gráfico Grad-CAM salvo em: {save_path}")
    plt.show()

print("✓ Visualização dos resultados Grad-CAM concluída.")

## Final Task

### Subtask:
Discutir as informações adicionais que o Grad-CAM fornece para a compreensão do diagnóstico e como ele pode ser usado para interpretar as decisões do modelo.


## Summary:

### Q&A
Grad-CAM provides visual information about which regions of an input image are most influential in the model's decision-making process. In the context of anomaly detection using a Generator model, Grad-CAM highlights areas within the image that the Generator found most challenging to reconstruct, specifically by identifying where the L1 loss (difference between the generated and original image) was highest. This allows for interpreting the model's decisions by visually pinpointing potential anomalous regions.

### Data Analysis Key Findings
*   A `GradCAM` class was successfully implemented, equipped with forward and backward hooks to capture feature maps and gradients from a specified target convolutional layer (`gerador.up7.model[0]`) of the Generator model.
*   During the application of Grad-CAM, an initial `RuntimeError` occurred due to the `feature_maps` tensor not having `requires_grad=True`. This was corrected by explicitly setting `output.requires_grad_(True)` within the `_save_feature_maps_hook` method.
*   Post-correction, Grad-CAM heatmaps were successfully generated for both a healthy (`leaf a21-a23 ad_1.png`) and a diseased (`a1001-1003 ad_0.png`) leaf image, along with the model's reconstructed outputs.
*   Visualization plots were successfully generated and saved, presenting a four-panel view for each image, including: the original image, the model's generated (reconstructed) image, the Grad-CAM heatmap, and the heatmap overlaid on the original image.

### Insights or Next Steps
*   Grad-CAM successfully provides a visual explanation of where the generative model "struggles" to reconstruct the image, indicating areas of potential anomaly. This can be crucial for localizing plant diseases.
*   The generated Grad-CAM heatmaps can be further analyzed by correlating the highlighted regions with expert-annotated disease areas to quantitatively assess the model's ability to localize anomalies.
